# Transition states
In this notebook we will locate a transition state by using a workflow which 1) Optimizes the reactant 2) performs a scan of one bond distance involved in the reaction 3) Takes the highest energy, computes the Hessian and optimizes the transition state.

In [ ]:
import veloxchem as vlx

In [ ]:
molecule = vlx.Molecule.read_smiles('c1ccccc1C(=O)C=CO')
molecule.set_dihedral_in_degrees([8,7,9,10], 0)
molecule.set_dihedral_in_degrees([7,9,10,11], 0)
molecule.set_dihedral_in_degrees([9,10,11,19], 0)
molecule.show(atom_indices=True)

In [ ]:
basis = vlx.MolecularBasis.read(molecule, 'def2-svp')

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.xcfun = 'blyp'
scf_drv.ri_coulomb = True
scf_results = scf_drv.compute(molecule, basis)

opt_drv = vlx.OptimizationDriver(scf_drv)
opt_drv.conv_energy = 1e-04
opt_drv.conv_drms = 1e-02
opt_drv.conv_dmax = 2e-02
opt_drv.conv_grms = 4e-03
opt_drv.conv_gmax = 8e-03
opt_results = opt_drv.compute(molecule, basis, scf_results)

In [ ]:
opt_drv.show_convergence(opt_results)

In [ ]:
molecule = vlx.Molecule.read_xyz_string(opt_results['final_geometry'])
molecule.show(atom_indices=True)

In [ ]:
opt_drv.constraints = ["scan distance 11 19 1.1 1.4 4"]

scan_results = opt_drv.compute(molecule, basis, scf_results)

In [ ]:
m = max(scan_results['scan_energies'])
index_max = scan_results['scan_energies'].index(m)

In [ ]:
ts_guess = vlx.Molecule.read_xyz_string(scan_results['scan_geometries'][index_max])
ts_guess.show()

In [ ]:
basis_ts = vlx.MolecularBasis.read(ts_guess, 'def2-svp')
scf_results_ts = scf_drv.compute(ts_guess, basis_ts)

ts_drv = vlx.OptimizationDriver(scf_drv)
ts_drv.transition = True
ts_results = ts_drv.compute(ts_guess, basis_ts, scf_results_ts)

In [ ]:
ts_drv.show_convergence(ts_results)

In [ ]:
activation_energy = ts_results['opt_energies'][-1]-opt_results['opt_energies'][-1]
print(f"Activation energy: {2625.5*activation_energy:.2f} kJ/mol")